<a href="https://colab.research.google.com/github/flying-smoothly/AI_vision/blob/master/7_15gpt_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip uninstall openai

In [2]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.1/337.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [3]:
!pip install python-dotenv

In [4]:
pip install opencv-python
pip install opencv-python-headless

SyntaxError: invalid syntax (<ipython-input-4-75d6990a88f1>, line 1)

In [5]:
import os
import cv2
import openai
from openai import OpenAI
import requests
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import io
from io import BytesIO
import base64
from dotenv import load_dotenv, find_dotenv

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
load_dotenv('/content/drive/MyDrive/key.env')

openai.api_key = os.getenv('OPENAI_API_KEY') #os 모듈을 사용하여 환경 변수에서 API 키를 불러옵니다.

if openai.api_key:
    print("API 키가 성공적으로 설정되었습니다.")
else:
    print("API 키를 설정하는 데 실패했습니다.")

client=OpenAI()

API 키가 성공적으로 설정되었습니다.


In [8]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "/content/drive/MyDrive/depth.bag/depth_Depth_31135.png"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {openai.api_key}"
}

payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "png 파일에 depth camera로 찍은 이미지의 깊이 정보와 RGB 정보가 저장되어 있어. 이미지는 스프레이를 찍은 거고 나는 스프레이의 깊이 정보를 알고 싶어. 스프레이가 카메라로부터 얼마나 떨어져 있었는지 png 파일을 읽어서 말해줄 수 있어? 얼마나 떨어져 있는지만 말해주면 돼."
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-9rg4DJYk3y8VkiHCbTqgslZ9jXqJQ', 'object': 'chat.completion', 'created': 1722580117, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '제공된 이미지는 깊이 카메라를 사용하여 촬영된 것으로 보이며, 색상은 심도 정보를 나타내는 데 사용됩니다. 일반적으로 이러한 이미지는 색상 맵을 사용하여 걸린 근사 거리를 나타냅니다. 이미지의 색상은 다음과 같은 일반적인 깊이 정보 매핑을 따를 수 있습니다:\n- 빨간색: 멀리 있는 물체\n- 파란색: 가까이 있는 물체\n\n이미지에서 스프레이의 위치를 명확히 알 수 없지만, 전체적으로 이미지의 중앙 부분이 녹색과 노란색으로 보이며 이는 중간 정도의 거리를 나타낼 수 있습니다. 반면, 하단의 파란색 부분은 카메라에 매우 가까운 거리를, 상단의 빨간색 부분은 매우 먼 거리를 나타냅니다.\n\n따라서, 스프레이가 무엇인지, 어떤 색으로 나타나는지 확실하지 않기 때문에 구체적인 거리를 알 수 없습니다. 이미지에서 중앙 영역의 물체가 일반적으로 중간 거리임을 감안할 때, 대략적인 거리를 추정할 수 있습니다.\n\n정확한 거리를 측정하려면 심도 카메라의 보정 데이터와 매핑 정보가 필요합니다. 이는 심도 이미지 내 픽셀의 색상 값과 실제 거리를 매핑한 데이터입니다. 만약 해당 정보를 알고 있다면'}, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 343, 'completion_tokens': 300, 'total_tokens': 643}, 'system_fingerprint': 'fp_bc2a86f5f5'}


In [ ]:
#####################################################
##               Read bag from file                ##
#####################################################


# First import library
import pyrealsense2 as rs
# Import Numpy for easy array manipulation
import numpy as np
# Import OpenCV for easy image rendering
import cv2
# Import argparse for command-line options
import argparse
# Import os.path for file path manipulation
import os.path

# Create object for parsing command-line options
parser = argparse.ArgumentParser(description="Read recorded bag file and display depth stream in jet colormap.\
                                Remember to change the stream fps and format to match the recorded.")
# Add argument which takes path to a bag file as an input
parser.add_argument("-i", "--input", type=str, help="Path to the bag file")
# Parse the command line arguments to an object
args = parser.parse_args()
# Safety if no parameter have been given
if not args.input:
    print("No input paramater have been given.")
    print("For help type --help")
    exit()
# Check if the given file have bag extension
if os.path.splitext(args.input)[1] != ".bag":
    print("The given file is not of correct file format.")
    print("Only .bag files are accepted")
    exit()
try:
    # Create pipeline
    pipeline = rs.pipeline()

    # Create a config object
    config = rs.config()

    # Tell config that we will use a recorded device from file to be used by the pipeline through playback.
    rs.config.enable_device_from_file(config, args.input)

    # Configure the pipeline to stream the depth stream
    # Change this parameters according to the recorded bag file resolution
    config.enable_stream(rs.stream.depth, rs.format.z16, 30)

    # Start streaming from file
    pipeline.start(config)

    # Create opencv window to render image in
    cv2.namedWindow("Depth Stream", cv2.WINDOW_AUTOSIZE)

    # Create colorizer object
    colorizer = rs.colorizer()

    # Streaming loop
    while True:
        # Get frameset of depth
        frames = pipeline.wait_for_frames()

        # Get depth frame
        depth_frame = frames.get_depth_frame()

        # Colorize depth frame to jet colormap
        depth_color_frame = colorizer.colorize(depth_frame)

        # Convert depth_frame to numpy array to render image in opencv
        depth_color_image = np.asanyarray(depth_color_frame.get_data())

        # Render image in opencv window
        cv2.imshow("Depth Stream", depth_color_image)
        key = cv2.waitKey(1)
        # if pressed escape exit program
        if key == 27:
            cv2.destroyAllWindows()
            break

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# 이미지 읽기
image_path = '/content/drive/MyDrive/LLM/cat6.jpeg'
image = cv2.imread(image_path)

# 이미지가 제대로 로드되었는지 확인
if image is None:
    print("Error: Unable to load image. Please check the file path.")
else:
    # 예시로 설정한 컵과 손의 바운딩 박스 좌표 (직접 정의 필요)
    cat_box = (96, 90, 247, 426)  # 고양이 바운딩 박스 좌표
    #hand_box = (830, 700, 960, 820)  # 손 바운딩 박스 좌표

    # 바운딩 박스 그리기
    cv2.rectangle(image, (cat_box[0], cat_box[1]), (cat_box[2], cat_box[3]), (0, 255, 0), 2)
    #cv2.rectangle(image, (hand_box[0], hand_box[1]), (hand_box[2], hand_box[3]), (255, 0, 0), 2)

    # 결과 이미지 보여주기
    cv2_imshow(image)

In [ ]:
# 이미지를 읽어옵니다
image_path = '/content/drive/MyDrive/LLM/cat1.jpg'
image = cv2.imread(image_path)

# 이미지가 제대로 로드되었는지 확인합니다
if image is None:
    raise ValueError(f"이미지를 로드할 수 없습니다: {image_path}")

# 이미지를 그레이스케일로 변환합니다
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 노이즈 제거를 위해 Gaussian 블러링을 적용합니다
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# 이진화하여 컵을 추출합니다
_, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# 모폴로지 연산을 사용하여 객체를 확장합니다
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
morphed = cv2.dilate(thresh, kernel, iterations=2)

# 외곽선을 찾습니다
contours, _ = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 각 외곽선을 감싸는 직사각형을 계산하고 이미지에 그립니다
for i, contour in enumerate(contours):
    x, y, w, h = cv2.boundingRect(contour)
    # 필터링 조건: 특정 크기 이상의 컨투어만 처리합니다
    if w > 50 and h > 50:
        # 바운딩 박스를 약간 확장합니다
        padding = 10
        x, y, w, h = x - padding, y - padding, w + 2 * padding, h + 2 * padding

        # 정규화된 좌표 출력
        print(f"Cup {i+1}:")
        print(f"Top-left: ({x}, {y})")
        print(f"Top-right: ({x+w}, {y})")
        print(f"Bottom-left: ({x}, {y + h})")
        print(f"Bottom-right: ({x + w}, {y + h})")
        print()

        # 원본 이미지에 사각형 그리기
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# 이미지를 출력합니다
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')  # 축을 끕니다
plt.show()